### ⚖️ SARSA vs Q-Learning


- **SARSA** (*on-policy*) – agentul învață pe baza acțiunilor reale efectuate, deci politica sa este mai precaută și tinde să evite riscurile.  
- **Q-Learning** (*off-policy*) – agentul învață presupunând că va alege întotdeauna cea mai bună acțiune posibilă, ceea ce duce la o politică mai agresivă, dar potențial mai eficientă.

Pe mediul **CliffWalking-v0**, diferența devine evidentă:  
SARSA preferă trasee sigure departe de prăpastie,  
în timp ce Q-Learning riscă mai mult pentru a ajunge mai repede la obiectiv.


In [ ]:
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt

env = gym.make("CliffWalking-v0")

alpha = 0.1      # learning rate
gamma = 0.99     # discount factor
epsilon = 1.0
eps_min = 0.01
eps_decay = 0.995
episodes = 500

n_states = env.observation_space.n
n_actions = env.action_space.n

def epsilon_greedy(Q, s, epsilon):
    if np.random.rand() < epsilon:
        return np.random.randint(n_actions)
    return np.argmax(Q[s])

def run_agent(algorithm="qlearning"):
    Q = np.zeros((n_states, n_actions))
    rewards = []

    global epsilon
    epsilon = 1.0

    for ep in range(episodes):
        s, _ = env.reset()
        a = epsilon_greedy(Q, s, epsilon)
        done = False
        total = 0

        while not done:
            s2, r, done, truncated, _ = env.step(a)
            a2 = epsilon_greedy(Q, s2, epsilon)
            total += r

            if algorithm == "sarsa":
                Q[s, a] += alpha * (r + gamma * Q[s2, a2] * (not done) - Q[s, a])
            elif algorithm == "qlearning":
                Q[s, a] += alpha * (r + gamma * np.max(Q[s2]) * (not done) - Q[s, a])

            s, a = s2, a2

        epsilon = max(eps_min, epsilon * eps_decay)
        rewards.append(total)

    return Q, rewards

Q_sarsa, r_sarsa = run_agent("sarsa")
Q_ql, r_ql = run_agent("qlearning")


window = 100
plt.plot(np.convolve(r_sarsa, np.ones(window)/window, mode='valid'), label="SARSA")
plt.plot(np.convolve(r_ql, np.ones(window)/window, mode='valid'), label="Q-Learning")
plt.xlabel("Episod")
plt.ylabel("Reward mediu (100 episoade)")
plt.legend()
plt.title("Comparatie SARSA vs Q-Learning - CliffWalking")
plt.show()


In [ ]:
def test_agent(Q, episodes=1, render=True):
    for ep in range(episodes):
        s, _ = env.reset()
        done = False
        total = 0
        steps = 0

        while not done:
            a = np.argmax(Q[s])
            s2, r, done, truncated, _ = env.step(a)
            total += r
            steps += 1
            s = s2

            if render:
                env.render()
        print(f"Episod {ep+1} -> Reward total: {total}, Pași: {steps}")


In [ ]:
print("=== Test SARSA ===")
test_agent(Q_sarsa, episodes=3, render=True)

print("\n=== Test Q-Learning ===")
test_agent(Q_ql, episodes=3, render=True)

In [ ]:
def trajectory(Q):
    s, _ = env.reset()
    done = False
    path = np.zeros(env.observation_space.n)
    while not done:
        path[s] += 1
        a = np.argmax(Q[s])
        s, r, done, truncated, _ = env.step(a)
    return path.reshape((4, 12))  # dimensiunea grilei pentru CliffWalking

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,4))
sns.heatmap(trajectory(Q_sarsa), annot=False, cmap="Blues", cbar=False)
plt.title("Traiectorie SARSA")
plt.show()

plt.figure(figsize=(10,4))
sns.heatmap(trajectory(Q_ql), annot=False, cmap="Reds", cbar=False)
plt.title("Traiectorie Q-Learning")
plt.show()
